In [ ]:
!{'python3 -m pip install dataflows==0.0.58 python-slugify'}
!{'python3 -m pip install -e git+https://github.com/OriHoch/dataflows-xlsx.git@master#egg=dataflows-xlsx'}

## Load the new groups from csv files

In [1]:
from dataflows import Flow, load, printer, update_resource

data = Flow(
    load('./data/new groups for Odata - open new item and generate random URL.csv'),
    load('./data/new groups for Odata - open new item and use ID for URL.csv'),
#     printer(tablefmt='html', num_rows=1)
).results()

new_groups = []
for resource in data[0]:
    for row in resource:
        new_groups.append({
            'tags': row['tags'],
            'id_tag': row['id tag'],
            'id': str(row['ID']) if row.get('ID') else '',
            'names': row.get('NAMES'),
            'name': row.get('name (should based on ID)'),
            'existing_group': row.get('existing group'),
            'entity_main_type': row['entity main type'],
            'entity_secondary_type': row.get('entity secondary type', row.get('entity secondary type (should based on ID)')),
            'entity_description': row['entity description'],
        })

Flow(
    (group for group in new_groups),
    update_resource('.*', name='new-groups', path='new-groups.csv'),
    printer(tablefmt='html', num_rows=2)
).process()

#,tags (string),id_tag (string),id (string),names (string),name (any),existing_group (any),entity_main_type (string),entity_secondary_type (string),entity_description (any)
1,אבי ברגר,b9af6db1-d9dc-4bc9-aed2-bcaa87b3dc88,None,אבי ברגר,None,None,Person,public servant,None
2,אבי גבאי,e27e32e6-c45b-499f-8513-c6c698850941,None,אבי גבאי,None,None,Person,public servant,None
3,אבי דזדק,d6c646f7-88e3-4753-b1c8-ddde95b1f09d,None,אבי דז'דק,None,None,Person,public servant,None
4,אבי כהן,6b12eff7-279e-4c69-8bc7-c4b5aac12330,None,אבי כהן,None,None,Person,public servant,None
5,אבי ליכט,c9cf6e6b-99e8-43b6-82a6-840eb30544a8,None,אבי ליכט,None,None,Person,public servant,None
6,אבי שלו,eba0f3c4-e4d4-4f61-b8d5-37270bde4ef0,None,אבי שלו,None,None,Person,None,None
...,,,,,,,,,
8,אביבה יצחקי,98905e0e-7838-4bed-8a9d-38e257c6c489,None,אביבה יצחקי,None,None,Person,public servant,None
9,אביגדור קלגסבלד,6e62c60c-e805-4dc5-a11e-ee0441482e0a,None,אביגדור קלגסבלד,None,None,Person,None,None
10,אביגדור קפלן,da778544-9a2f-46f4-9e93-9b3ef0ad6745,None,אביגדור קפלן,None,None,Person,public servant,None


(<datapackage.package.Package at 0x7fc981a01668>, {})

## Load budgetkey entities and people

In [ ]:
from dataflows import dump_to_path

Flow(
    load('https://next.obudget.org/datapackages/entities/scored/datapackage.json'),
    dump_to_path('./data/budgetkey-entities'),
    printer(tablefmt='html', num_rows=1)
).process()

In [63]:
import csv

csv.field_size_limit(csv.field_size_limit()*100)

Flow(
    load('https://next.obudget.org/datapackages/people/aggregated/datapackage.json'),
    dump_to_path('./data/budgetkey-people'),
    printer(tablefmt='html', num_rows=1)
).process()

#,details (array),key (string),score (number),person_key (string)
1,"[{'company': 'אנ אסטיטס וי יורופיאן סנטראל', 'doc_id': None, 'event': 'appointment', 'full_name': '- ...",- -,1,- -
2,"[{'company': 'אסטיטס בי וי יורופיאן סנטראל', 'doc_id': None, 'event': 'appointment', 'full_name': '_ ...",_________,1,_________
...,,,,
33462,"[{'company': 'החזקות מבטחים מנורה', 'doc_id': None, 'event': 'appointment', 'full_name': 'תפוחי ישרא ...",תפוחי,1,תפוחי


(<datapackage.package.Package at 0x7fb9c9550c18>,
 {'count_of_rows': 33462,
  'bytes': 91518375,
  'hash': '0422d90e403a29961eea6ee01ba426d8',
  'dataset_name': None})

In [65]:
budgetkey_entities_by_id = {}

Flow(
    load('./data/budgetkey-entities/datapackage.json'),
    lambda row: budgetkey_entities_by_id.update(**{row['id']: row})
).process()

(<datapackage.package.Package at 0x7fb9c93b5780>, {})

## update entity (corporations) groups

In [3]:
import getpass
CKAN_API_KEY = getpass.getpass("CKAN_API_KEY")

CKAN_API_KEY ····································


In [ ]:
import requests, json

entity_groups_to_create = {}

for group in new_groups:
    if group.get('id'):
        entity = budgetkey_entities_by_id.get(str(group['id']))
        if entity:
            group['tags'] = entity['name'].replace('~', '"')
            group['entity_secondary_type'] = entity['kind_he']
        else:
            print('WARNING!')
            print('{}: missing related budgetkey entity: {}'.format(group['id'], group['tags']))
        entity_groups_to_create.setdefault(group['id'], []).append({
            '__group': group,
            'name': group['id'],
            'title': group['tags'],
            'extras': [
                {'key': key, 'value': value}
                for key, value in {
                    **{'budgetkey_entity_'+ek: ev for ek, ev in (entity['details'].items() if entity else [])},
                    'budgetkey_entity_id': group['id'],
                    'budgetkey_entity_kind': entity['kind'] if entity else '',
                    'budgetkey_entity_kind_he': entity['kind_he'] if entity else '',
                    'entity_main_type': group['entity_main_type'],
                    'entity_secondary_type': group['entity_secondary_type'],
                    'entity_description': group['entity_description']
                }.items()
            ]
        })

for group_id, groups in entity_groups_to_create.items():
    group_json = groups[0]
    if group_json['name'] == '520036435': continue
    id_tags = set()
    for group in [g['__group'] for g in groups]:
        id_tag = group.get('id_tag')
        if id_tag:
            id_tags.add(str(id_tag))
    group_json['extras'].append({'key': 'import_id_tags', 'value': ','.join(id_tags)})
    for extra in group_json['extras']:
        if isinstance(extra['value'], list):
            extra['value'] = json.dumps(extra['value'], ensure_ascii=False)
        elif extra['value'] is None:
            extra['value'] = ''
        else:
            extra['value'] = str(extra['value'])        
    res = requests.post(
        'https://www.odata.org.il/api/3/action/group_create', 
        json=group_json,
        headers={'Authorization': CKAN_API_KEY}
    ).json()
    # print(res)
    if res['success']:
        print('new corporation group created: {} / {}'.format(group_json['name'], group_json['title']))
#         break
    elif res.get('error', {}).get('name') == ['שם רשות קיים כבר בבסיס הנתונים']:
        print('updating existing entity: {} / {}'.format(group_json['name'], group_json['title']))
        res = requests.post(
            'https://www.odata.org.il/api/3/action/group_update', 
            json={'id': group_json['name'], **group_json},
            headers={'Authorization': CKAN_API_KEY}
        ).json()
        # print(res)
        # break
    else:
        print('Unexpected response for CKAN group_create:')
        print(group_json)
        print(res)
        break;


## Update groups without id

In [ ]:
import requests, json
from slugify import slugify

groups_to_create = {}

for group in new_groups:
    if not group.get('id'):
        group_slug = slugify(group['tags'])
        groups_to_create.setdefault(group_slug, []).append({
            '__group': group,
            'name': group_slug,
            'title': group['tags'],
            'extras': [
                {'key': key, 'value': value}
                for key, value in {
                    'entity_main_type': group['entity_main_type'],
                    'entity_secondary_type': group['entity_secondary_type'],
                    'entity_description': group['entity_description']
                }.items()
            ]
        })

for group_slug, groups in groups_to_create.items():
    group_json = groups[0]
    id_tags = set()
    for group in [g['__group'] for g in groups]:
        id_tag = group.get('id_tag')
        if id_tag:
            id_tags.add(str(id_tag))
    group_json['extras'].append({'key': 'import_id_tags', 'value': ','.join(id_tags)})
    for extra in group_json['extras']:
        if isinstance(extra['value'], list):
            extra['value'] = json.dumps(extra['value'], ensure_ascii=False)
        elif extra['value'] is None:
            extra['value'] = ''
        else:
            extra['value'] = str(extra['value'])        
    res = requests.post(
        'https://www.odata.org.il/api/3/action/group_create', 
        json=group_json,
        headers={'Authorization': CKAN_API_KEY}
    ).json()
    # print(res)
    if res['success']:
        print('new group created: {} / {}'.format(group_json['name'], group_json['title']))
#         break
    elif res.get('error', {}).get('name') == ['שם רשות קיים כבר בבסיס הנתונים']:
        print('updating existing group: {} / {}'.format(group_json['name'], group_json['title']))
        res = requests.post(
            'https://www.odata.org.il/api/3/action/group_update', 
            json={'id': group_json['name'], **group_json},
            headers={'Authorization': CKAN_API_KEY}
        ).json()
        if not res['success']:
            print(res)
            break
    else:
        print('Unexpected response for CKAN group_create:')
        print(group_json)
        print(res)
        break;


## import id_tags

In [4]:
import requests

all_group_names = requests.get('https://www.odata.org.il/api/3/action/group_list', headers={'Authorization': CKAN_API_KEY}).json()['result']

In [ ]:
def add_tag_datasets_to_group(tag_id, group_name):
    res = requests.get(
        'https://www.odata.org.il/api/3/action/tag_show?id={}&include_datasets=True'.format(tag_id), 
        headers={'Authorization': CKAN_API_KEY}
    ).json()
    assert res['success']
    tag = res['result']
#     print(tag)
    for package in tag['packages']:
        package_id = package['id']
        res = requests.get(
            'https://www.odata.org.il/api/3/action/package_show?id={}&include_datasets=True'.format(package_id), 
            headers={'Authorization': CKAN_API_KEY}
        ).json()
        assert res['success']
        package = res['result']
        if group_name not in [g['name'] for g in package['groups']]:
            package['groups'].append({'name': group_name})
            res = requests.post(
                'https://www.odata.org.il/api/3/action/package_update',
                json=package,
                headers={'Authorization': CKAN_API_KEY}
            ).json()
            assert res['success']
            print('updated package: https://www.odata.org.il/dataset/{}'.format(package_id))


def add_datasets_by_name_to_group(name, group_name):
    res = requests.get(
        'https://www.odata.org.il/api/3/action/package_search?q={}'.format(name), 
        headers={'Authorization': CKAN_API_KEY}
    ).json()
    assert res['success']
    for result in res['result']['results']:
        package_id = result['id']
        res = requests.get(
            'https://www.odata.org.il/api/3/action/package_show?id={}&include_datasets=True'.format(package_id), 
            headers={'Authorization': CKAN_API_KEY}
        ).json()
        assert res['success']
        package = res['result']
        if group_name not in [g['name'] for g in package['groups']]:
            package['groups'].append({'name': group_name})
            res = requests.post(
                'https://www.odata.org.il/api/3/action/package_update',
                json=package,
                headers={'Authorization': CKAN_API_KEY}
            ).json()
            assert res['success']
            print('updated package: https://www.odata.org.il/dataset/{}'.format(package_id))
            

for group_name in all_group_names:
    group = requests.get(
        'https://www.odata.org.il/api/3/action/group_show?id={}&include_extras=true&include_users=False&include_datasets=False&include_followers=False'.format(group_name), 
        headers={'Authorization': CKAN_API_KEY}
    ).json()['result']
    for extra in group['extras']:
        if extra['key'] == 'import_id_tags':
            for tag_id in extra['value'].split(','):
                if tag_id.strip() == 'no tag': continue
                if tag_id.strip() == 'name of dataset':
                    add_datasets_by_name_to_group(group['title'], group_name)
                else:
                    add_tag_datasets_to_group(tag_id, group_name)